In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データの分析を行うために、以下の主要な観点から傾向を検討します。

### 1. 売上のカテゴリー別分析
- データには「食品」「衣服」「日用品」というカテゴリがあります。それぞれのカテゴリーの売上総額、販売数、利益（売上 - 原価）を算出することで、どのカテゴリが最も収益性が高いかを評価します。

### 2. 商品別のパフォーマンス
- 各商品コードごとに売上、数量、原価を集計し、売上高の上位商品や利益率の高い商品を特定します。

### 3. 時間軸の分析
- 売上日のデータを基に、月別売上のトレンドを分析します。これにより、特定の月や季節に売上が増加するパターンを見つけ出すことができます。

### 4. 単価と数量の相関
- 商品ごとの単価と数量の関係を調査し、単価が高い商品が大量に売れるのか、逆に単価が低い商品が大量に売れるのかを見ます。

### 5. 利益率の評価
- 各商品の利益率を計算し、高利益率の商品に焦点を当て、今後の在庫やマーケティング策略の策定に役立てます。

### 結果の例
以下は、分析から得られる可能性のあるインサイトの例です。

- **カテゴリー別売上分析結果**: 
  - 食品の売上が全体の40%を占めていることがわかり、成長の余地が大いにある。特に、トレンド商品（例：ザクロやココナッツ）は需要が高まる可能性がある。
  
- **商品パフォーマンス**: 
  - Tシャツやジーンズが売上高で上位にランクイン。特にジーンズは高単価だが、利益率も期待できるため重点的なプロモーションを考える。

- **時間軸分析**:
  - 1月は季節性商品の販売が低かったが、4月に入ってから回復してきている。母の日やゴールデンウィークに向けてプロモーション戦略を強化する余地がある。

- **相関分析**: 
  - 高単価の衣服カテゴリーは数量が少ないが、利益が大きいことが多い。現状の在庫回転率を考慮し、在庫管理を行うことが重要。

- **利益率評価**: 
  - 売上単価が高い「レインパンツ」や「レインシューズ」は利益率も高いことから、リピートオーダーを視野に入れたマーケティングが考えられる。

### 結論
この売上データから、製品ラインナップの最適化、マーケティング戦略の見直し、特定のターゲット市場への焦点を当てた在

In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                この売上データの分析を行うために、以下の主要な観点から傾向を検討します。
1                                                    
2                                  ### 1. 売上のカテゴリー別分析
3   - データには「食品」「衣服」「日用品」というカテゴリがあります。それぞれのカテゴリーの売上...
4                                                    
5                                  ### 2. 商品別のパフォーマンス
6    - 各商品コードごとに売上、数量、原価を集計し、売上高の上位商品や利益率の高い商品を特定します。
7                                                    
8                                       ### 3. 時間軸の分析
9   - 売上日のデータを基に、月別売上のトレンドを分析します。これにより、特定の月や季節に売上が...
10                                                   
11                                    ### 4. 単価と数量の相関
12  - 商品ごとの単価と数量の関係を調査し、単価が高い商品が大量に売れるのか、逆に単価が低い商品...
13                                                   
14                                      ### 5. 利益率の評価
15  - 各商品の利益率を計算し、高利益率の商品に焦点を当て、今後の在庫やマーケティング策略の策定...
16                                                   
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
